![Podmanlogo](Pictures/podman-logo.png)

# From Podman to Kubernetes and viceversa

When working with kubernetes what most users want is a way of easily deploying in kubernetes what they have already tested in Podman. This can be done easily as Podman helps generating a yaml file that can be used afterwards in kubernetes. We demonstrated this on the Podman 101 workshop, but we are going to do it again with our Patient Portal application. Then we'll deploy it to an actual kubernetes cluster (in our case we chose OpenShift distribution).

Lets begin by creating all the containers of our application. As alwasy, you need to start by creating the networks:

In [ ]:
podman network create database
podman network create payment

Now we can spin up all of our three containers. We added a sleep in the command list to give enough time to the database to initialize before spining up the frontend container, otherwise the deployment of the app would fail.

In [ ]:
podman run -d --rm --name database --network database quay.io/skupper/patient-portal-database
sleep 10
podman run -d --rm --name payment-processor --network payment quay.io/skupper/patient-portal-payment-processor
podman run -d --rm --name frontend --network payment,database -p 8080:8080 \
-e DATABASE_SERVICE_HOST="database" \
-e DATABASE_SERVICE_PORT="5432" \
-e PAYMENT_PROCESSOR_SERVICE_HOST="payment-processor" \
-e PAYMENT_PROCESSOR_SERVICE_PORT="8080" \
quay.io/skupper/patient-portal-frontend

Note that we're not adding any volume, this is because we don't have a persistent storage provider for our OpenShift cluster, hence we generate containers with ephemeral storage. Nonetheless it's important to remark that this whole process can be replicated using volumes as well.

Check everything is working as expected:

In [ ]:
podman ps -a

Check the logs:

In [ ]:
podman logs frontend

Run an http request:

In [ ]:
curl -s localhost:8080

Everything looks fine, it's time to create a yaml file to be used in kubernetes out of our running containers. For that we can simply use the "podman generate kube" command. Run it for the database container and review the output:

In [ ]:
podman generate kube --replicas 1 --service --type deployment database > database.yml
cat database.yml

See that the yaml file contains two sections separated by "---", this is because we have two object definitions in the same file. The first one defines the Service object and the second one the Deployment. Podman is generating this specific objects in the file because we specified the option "--service" for adding a service to the output and "--type deployment" to specify we want to use a Deployment, otherwise it would have just defined a pod. Moreover you can define the number of replicas of your container you want to have.

As we are pointing the command "podman generate kube" command to a single container it will consider that we want to create a pod with a single container in our kubernetes cluster. We could also point to a Podman pod and it would generate a deployment with multiple containers in a pod if that is what we had in the Podman pod.

Continue with the payment-processor container:

In [ ]:
podman generate kube --replicas 3 --service --type deployment payment-processor > payment-processor.yml
cat payment-processor.yml

See how we specified we want three replicas of this service and, in the yaml file, it's also defined under the "spec.replicas" part of the Deployment definition.

And, last, for the frontend:

In [ ]:
podman generate kube --replicas 1 --service --type deployment frontend > frontend.yml
cat frontend.yml

Now, lets deploy all these workloads to our kubernetes cluster.

~~~
SECTION DEPLOYING THE WORKLOAD TO KUBERNETES, TO BE WRITTEN ONCE THE OPENSHIFT CLUSTER IS UP AND RUNNING
~~~

We've seen how you can use Podman to generate the files that you'll use later to deploy your workloads in kubernetes. And probably you're thinking that kubernetes yaml files are super cool and you'd like to use them also for Podman workloads. No worries, Podman got your back.

Before moving on stop all of the running pods:

In [ ]:
podman pod rm --all -f

Now, if you wanted to run your pod in Podman using the kubernetes yaml file you just need to run the following command pointing to your yaml file:

In [ ]:
podman kube play database.yml

If you check the output, it generates a pod for with the container within it. This is because in kubernetes we always have a pod and Podman just follows what's in the yaml file.

In [ ]:
podman ps -a --pod

Deploy the payment processor and the frontend using their kubernetes yaml files.

In [ ]:
podman kube play payment-processor.yml
podman kube play frontend.yml

First thing you notice is a warning. Podman's telling us that the amount of replicas for the payment-processor workload has been reduced to 1. This is because Podman is not designed for scalability and it's single node, therefore it doesn't make sense to have multiple replicas.

Review all containers are up and running:

In [ ]:
podman ps -a --pod

We have now 6 containers, this is because we have 3 pods and Podman deploys a pod infrastructure management container in every pod.

Review frontend logs (we need to use the automatically generated name for the container):

In [ ]:
podman logs frontend-pod-deployment-pod-frontend

Also check if the app receives http requests:

In [ ]:
curl -s localhost:8080

Everything is working perfectly and using yaml files!

There is one last integration between Podman and kubernetes yaml files that I want to mention. Remember Quadlets? They are pretty similar to kubernetes yaml files, you define in the your container and systemd deploys them. But isn't it duplication of work to have both, a Quadlet and a kubernetes yaml file, for the same container? It is! And that is why you can have Quadlets directly pointing to kubernetes yaml files instead of having to define the whole container.

Lets see how these Quadlets would look like. Start with the database one:

In [ ]:
cat << EOF > database.kube
[Install]
WantedBy=default.target

[Kube]
# Point to the yaml file in the same directory
Yaml=database.yml
EOF
cat database.kube

As you can see we just define the target, as always, and the we point to our yaml file in the "[Kube]" section. As easy as it is!
Rembember we are using a relative path to the yaml file.

Do the same for the payment processor container:

In [ ]:
cat << EOF > payment-processor.kube
[Install]
WantedBy=default.target

[Unit]
Requires=database.service
After=database.service

[Kube]
# Point to the yaml file in the same directory
Yaml=payment-processor.yml
EOF

We added the "[Unit]" section to guarantee everything is booted in the right order.

And for the frontend container:

In [ ]:
cat << EOF > frontend.kube
[Install]
WantedBy=default.target

[Unit]
Requires=payment-processor.service
After=payment-processor.service

[Kube]
# Point to the yaml file in the same directory
Yaml=frontend.yml
EOF

Move all the files to "~/.config/containers/systemd/". You need to move your yaml file as we used a relative path in the Quadlet definition.

In [ ]:
mv ~/* ~/.config/containers/systemd/

Now that all the files are present in the correct directory you just need to reload the systemctl daemon:

In [ ]:
systemctl --user daemon-reload

Finally, use systemctl to manage your containerized workload as a systemd unit:

In [ ]:
systemctl --user start database.service
systemctl --user start payment-processor.service
systemctl --user start frontend.service

Review all of your containers and pods are up and running:

In [ ]:
podman ps -a --pod

Check frontend logs:

In [ ]:
podman logs frontend-pod-deployment-pod-frontend

And make sure your app is ansering http requests:

In [ ]:
curl -s localhost:8080

As you can see everything works perfectly!

We have seen many ways in which Podman and Kubernetes are interconnected. Hopefully you can leverage all this interconnections between both technologies to have a better integration and usability in your cloud-native containerized environments.

# Clean up

In [ ]:
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f
rm -rf ~/.config/containers/systemd/*